### For conducting SCFT/inverse design for micelle structures of polymer solutions

Note: support for this function is limited

In [ ]:
import sys
import jax.numpy as jnp
import jax
import os

sys.path.append(os.path.abspath(".."))

from src.space_group import SpaceGroupCollection
from src.sequence import Sequence
from src.scft_micelle import SCFTMicelle
from src.utils import STRUCTURES_DIR, CONVERGED_STRUCTURES_DIR, PROJECT_ROOT, INITIAL_GUESS_DIR

jax.config.update("jax_enable_x64", True) # Enable 64-bit precision

**First run the SCFT code to get a target structure**

In [ ]:
### All this can be stored in a csv file and read line by line
# Define the files we want to load
space_groups_to_load = ['p_6_m_m_48_48_1', 'I_m_-3_m_32_32_32', 'I_a_-3_d_32_32_32', 'P_m_-3_m_32_32_32']#['P_42%m_n_m_36_36_24', 'P_63%m_m_c_24_24_36', 'P_6%m_m_m_24_24_36', 'C_m_m_m_32_32_32']  # Replace with actual list of space group names, e.g., ['P_-1_48_1_1']
sg_info_file = PROJECT_ROOT / "space_group_info.csv"

# Set up the space group geometries
sg_collection = SpaceGroupCollection.from_files(
    desired_sg=space_groups_to_load,
    sg_info_file=sg_info_file,
    structure_folder=STRUCTURES_DIR
)
# sg_collection contains associated info for each space group. For example, we can access sg_collection.space_groups['P_-1_48_1_1'].cell for the cell parameters
# sg_collection['P_-1_48_1_1'].u0['u_fwd'] access the forward u0 initialization for the space group P_-1_48_1_1
# sg_collection['P_-1_48_1_1'].data is a dictionary that allows access to loaded information files

# Note: It is necessary to modify the cell parameters for micelle simulations. Generally must be larger than equivalent structures for melts
#sg_collection['P_m_-3_m_32_32_32'].data['cell'] = 10.0

In [ ]:
# Set up the sequence
# Read a vector containing the magnitudes we want
#mag_vector = jnp.array(np.loadtxt(PROJECT_ROOT / "abc_vectors.csv", delimiter = ','))
#block_fractions = jnp.array([0.22, 0.56, 0.22])  # Example block fractions for diblock copolymer

chi = 25 #17.5
block_fractions = jnp.array([0.75, 0.25])
block_magnitudes = jnp.array([jnp.sqrt(chi) / 2, -jnp.sqrt(chi) / 2])  # Example magnitudes for the blocks, can be adjusted as needed
# Stack the block fractions and magnitudes into a single array
f_params = jnp.vstack([block_fractions, block_magnitudes])
seq_name = 'diblock_f75_c25_micelle_pm3m' # This is what the target structure is saved as
sequence = Sequence.generate(name=seq_name, basis='linear_basis', transform='block_copolymer', Ns = 125, Nbasis = 125, Np = 125, Ndim = 1, f_params = f_params)

# Initialize the SCFT object
scft = SCFTMicelle(space_group=sg_collection, sequence=sequence, c=0.1, solvent = jnp.sqrt(chi) / 2)

# Run SCFT with default settings
x_dict = {}
p_dict = {}
sg_list = ['P_m_-3_m_32_32_32']

# If loading
#load_dict = {'filename' : INITIAL_GUESS_DIR / 'sigma_psi.csv', 'cell': jnp.array([7.1, 3.5]), 'angle' : None}

for sg in sg_list:
    #sg = 'I_a_-3_d_32_32_32'
    x_scft, p_scft = scft.run_scft(sg = sg, load = False, save = True) # load, load_dict, save are options
    x_dict[sg] = x_scft
    p_dict[sg] = p_scft

**Inverse design towards the target structure you just solved for**

In [ ]:
import importlib
import src.inverse_design_micelle
importlib.reload(src.inverse_design_micelle)
from src.inverse_design_micelle import InverseDesignMicelle

# Initialize an Inverse Design SCFT object
c = 0.1 # This is the polymer volume fraction
sequence_id = Sequence.generate(name='inverse_design', basis='linear_basis', transform='multiblock', Nbasis = 125, Ns = 125, Np = 2, Ndim = 3)

inverse_design = InverseDesignMicelle(sg_collection, sequence_id, c)

# Load a structure
sg = 'P_m_-3_m_32_32_32'
file = f'scft_{seq_name}_{sg}_1_phi.npy' # Choose 0 or 1 depending which structure you want to target. A sample target is supplied as 'id_target_pm-3m_micelle.npy'
target = inverse_design.load_target(sg = sg, target_file = file, target_folder = CONVERGED_STRUCTURES_DIR)

save_name = 'micelle_sphere_f75_pm3m'

inverse_design.run_all(target=target, save_name=save_name)